In [ ]:
list_todo()

##### Investigate null values in item_type_code_name

###### Cross Tabulation of scoring_type_code and response_correctness

In [ ]:
pdDf = dfInv.toPandas()
pd.crosstab(pdDf.learner_attempt_status.fillna('null'), pdDf.response_correctness.fillna('null'),)

In [ ]:
def testing():
  cols = ['assigned_item_status', 'ced_assignment_type_code', 'is_affecting_grade', 'is_force_scored', 'item_is_offline_scored', 'item_type_code_name',  'response_correctness', 'scoring_type_code']
  pdDf = dfInv.select(*cols).toPandas()
  # Plot features associations
  associations(pdDf)

testing()

###### Cross Tabulation of

In [ ]:
pdDf = dfInv.toPandas()
pd.crosstab(pdDf.assigned_item_status.fillna('null'),     pdDf.item_is_offline_scored.fillna('null'))

###### Cross Tabulation of

In [ ]:
pd.crosstab(pdDf.ced_assignment_type_code.fillna('null'), pdDf.is_affecting_grade.fillna('null'))

###### Cross Tabulation of

In [ ]:
pd.crosstab(pdDf.item_type_code_name.fillna('null'), pdDf.scoring_type_code.fillna('null'))

###### Can final_score_unweighted be used to predict response_correctness?

In [ ]:
# Count the number of each response_correctness level where the final_score_unweighted is 0
dfInv.filter(col("final_score_unweighted") == 0).groupBy("response_correctness").count().orderBy("count", ascending=False).show(50, False)

- final_score_unweighted be used to predict response_correctness
  - no
- is response_correctness null or incorrect if final_score_unweighted is 0
  - no

Are the following related?
- assignment_due_date, assignment_final_submission_date and assignment_start_date have 1566 empty dates
- assignment_attempt_number and assignment_max_attempts have 1566 records with the value 0
- there are 1566 records with the response_correctness "[unassigned]"

###### Cross-tabulation learner_attempt_status vs response_correctness

In [ ]:
import pandas as pd

dfPd = dfInv.toPandas()
# Return cross-tabulation table of learner_attempt_status vs response_correctness adding counts for null values
pd.crosstab(dfPd.learner_attempt_status.fillna('null'), dfPd.response_correctness.fillna('null'), margins=True, margins_name="Total")

###### Cross-tabulation learner_attempt_status vs assigned_item_status

In [ ]:
# Return cross-tabulation table of learner_attempt_status vs assigned_item_status adding counts for null values
pd.crosstab(dfPd.learner_attempt_status.fillna('null'), dfPd.assigned_item_status.fillna('null'), margins=True, margins_name="Total")

###### Cross-tabulation learner_attempt_status vs scoring_type_code

In [ ]:
# Return cross-tabulation table of learner_attempt_status vs scoring_type_code adding counts for null values
pd.crosstab(dfPd.learner_attempt_status.fillna('null'), dfPd.scoring_type_code.fillna('null'), margins=True, margins_name="Total")

###### Interval Correlations

In [ ]:
from pyspark.sql.functions import unix_timestamp, col
import seaborn as sn
import matplotlib.pyplot as plt

dfPd =  filter_default(dfInv).select(* (unix_timestamp(c).alias(c) for c in intervalVars) ).toPandas()

corrMatrix = dfPd.corr()
plt.figure(figsize=(10,12))
sn.heatmap(corrMatrix, annot=True)
plt.show()


Interval Correlation Results
- Perfectly correlated (1 or -1)
  - assignment_due_date and..
    - assignment_final_submission_date
    - assignment_start_date
    - Comments: Verify and possibly use only one
- Highly correlated (> .7)
  - max_student_start_datetime and min_student_stop_datetime
    - Explanation: quizes are short and ending always follows starting
  - scored_datetime and..
    - student_stop_datetime
    - was_fully_scored_datetime
    - was_submitted_datetime_actual
    - Explanation: actions closely follow stopping
  - student_start_datetime and..
    - student_stop_datetime
    - was_inprogress_datetime
    - Explanation: quizes are short and ending always follows starting
  - student_stop_datetime and..
    - scored_datetime
    - student_start_datetime
    - was_fully_scored_datetime
    - Explanation: quizes are short and ending always follows starting

In [ ]:
date_stats(dfInv, "assignment_due_date", "assignment_final_submission_date").describe().show()


# Investigations

### Is learner_attempt_status of "fully scored" perfectly correlated other variables

In [ ]:
cols = single_val(dfClean, "learner_attempt_status")
print("Correlated Vars", cols)

- No

In [ ]:
df = dfClean.filter(col("assignment_due_date").isNull() == True)
cols = single_val(df, "assignment_due_date")

# Show the results on two lines
length = len(cols)
middle_index = length//2
first_half = cols[:middle_index]
second_half = cols[middle_index:]

df.select(*first_half).show(1)
df.select(*second_half).show(1)



Yes they are all related
- only occurs in one organization
- could be record keeping differences
- EXCLUDE?

### Why Are Empty Dates Empty

In [ ]:
from pyspark.sql.functions import count, col
# Count Default / Empty Dates

for f in intervalVars:
  count = dfClean.filter(col(f).isNull() == True).count()
  print (f,"=", count)

In [ ]:
df = dfClean.filter(col("scored_datetime").isNull() == True)
cols = single_val(df, "scored_datetime")
df.select(*cols).show(1)


scored_datetime is null at times when fully scored

In [ ]:
import math

df = dfClean.filter(col("student_start_datetime").isNull() == True)
cols = single_val(df, "student_start_datetime")

# Show the results on two lines

df.select(*cols[:5]).show(1)
df.select(*cols[5:9]).show(1)
df.select(*cols[9:]).show(1)

- Null for one organization
- Could just be record keeping difference

In [ ]:
df = dfClean.filter(col("was_in_progress_datetime").isNull() == True)
cols = single_val(df, "was_in_progress_datetime")
df.select(*cols).show(1)


- Only when fully scored

In [ ]:
df = dfClean.filter(col("was_submitted_datetime_actual").isNull() == True)
cols = single_val(df, "was_submitted_datetime_actual")
df.select(*cols).show(1)


- Only when fully scored

### Reclassify categories in item_type_code_name

In [ ]:
# Before Categories
dfClean.select("item_type_code_name").distinct().orderBy("item_type_code_name").show(50, False)


Combine Suffix Levels
- The levels with the suffix Response (ex: FillinBlankResponse) is the same type of question as level without the suffix (ex: fillInTheBlank)

In [ ]:
# Combine fillInTheBlank and FillinBlankResponse
dfClean = dfClean.withColumn("item_type_code_name", when( col("item_type_code_name") == "FillinBlankResponse", "fillInTheBlank" ).otherwise(col("item_type_code_name")) )

# Combine multipleChoice and MultipleChoiceResponse
dfClean = dfClean.withColumn("item_type_code_name", when( col("item_type_code_name") == "MultipleChoiceResponse", "multipleChoice" ).otherwise(col("item_type_code_name")) )


In [ ]:
%sql
select assessment_instance_id,
 section_id,
 assessment_instance_attempt_id,
 assignment_start_date
FROM clean_data
WHERE assessment_instance_attempt_id IS NOT NULL
ORDER BY assessment_instance_id, assessment_instance_attempt_id, student_start_datetime LIMIT 100
;

The assignment start date is the same for every assessment_instance_id.  Looks like it's assigned with the instance is created

In [ ]:
%sql
SELECT COUNT(*) FROM clean_data WHERE assignment_start_date IS NULL;

Hmm... there's that 1566 again.  Let's look at those records

In [ ]:
dfDesc.select('field').show(40, False)

In [ ]:
save_dict(shared, 'shared2')
save_dict(dfInv, 'dfInv')